# Imports and paths

In [ ]:
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib as mpl
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
%matplotlib inline
mpl.rcParams.update({'font.size':10})
plasma_map = plt.cm.plasma(np.linspace(0,1,15))
kcolors = {
    'movement': plasma_map[12,:],
    'early': plasma_map[10,:],
    'late': plasma_map[8,:],
    'biphasic': plasma_map[5,:],
    'negative': plasma_map[2,:],
    'unresponsive': 'dimgrey'
}
tcolors = {
    'gaze': 'firebrick',
    'comp': 'mediumblue',
    'rc': 'indigo'
}

In [ ]:
psth_bins = np.arange(-200,401)

In [ ]:
savepath = '/home/niell_lab/Data/marmoset/gazeshift/panels/'
data = loadmat('/home/niell_lab/Data/marmoset/gazeshift/Pooled_V1Hart_Preload_Final.mat')
totdata = loadmat('/home/niell_lab/Data/marmoset/gazeshift/Pooled_V1Hart_Preload_TotalInfo.mat')

%ISACMOD = [];  % saccade modulation psths for images (each row a neuron)
                % time steps are 1ms, with 50 being saccade offset
%GSACMOD = [];  % saccade modulation psths for gratings (each row a neuron)
%SFTUNE = [];   % spatial freq tuning curves (each row a neuron)
%TFTUNE = [];   % temporal freq tuning (fft of temporal kernel), each row a neuron
%ORTUNE = [];   % orientation tuning per neuron
%BASEMU = [];   % baseline firing rate, to natural images (inclusion criteria?)
%PEAKIM = [];   % time of the peak latency of nat image modulation
%SFGRID = [];   % for each neuron, the spatial frequencies uses in RF 
%ANIMID = [];   % which of the two animals, 1=Allen, 2=Logan

# Rasters for example units

In [ ]:
totunits = totdata['TotalInfo']

In [ ]:
def mRaster(ax, rast, n=500):
    usetrials = np.array(sorted(np.random.choice(np.arange(0,int(np.max(rast[:,1]))), n, replace=False)))
    
    for row, tnum in enumerate(usetrials):
        sps = rast.copy()
        sps = sps[sps[:,1].astype(int)==tnum]
        sps = sps[:,0]
        
        ax.plot(sps, np.ones(sps.size)*row, '|', color='k', markersize=0.3)
        
    ax.set_xlim([-0.2,0.4])
    ax.set_ylim([n, 0])
    ax.set_yticks(np.linspace(0,n,3))
    
    ax.set_xticks(np.linspace(-.2,.4,4))
    ax.set_xticklabels([])
    # ax.set_xticklabels(np.linspace(-200,400,4).astype(int))
    
    return useinds

In [ ]:
example_units = [9, 96, 30]
# [39, 21, 95] [9, 30]
# 9 is super strong early cell

fig, axs = plt.subplots(3,3, figsize=(6,4.8), dpi=300)

for uPos, uNum in enumerate(example_units):

    unitlabels = ['tagname','pathname','pathplot','isolation','depth','duration',
                  'waveform','channel','shank','ISI','Hart','SacGrating','SacImage']
    unit_dict = dict(zip(unitlabels, list(totunits[uNum][0][0][0])))

    sacimlabels = ['EventList','StimWin','TrialType','StimRast','OriInd','StimTT',
                   'StimUU','StimSU','OriInd2','StimRast2','StimTT2','StimUU2',
                   'StimSU2','BaseMu','BaseMu2']
    sacim_dict = dict(zip(sacimlabels, list(unit_dict['SacImage'][0][0])))


    hartlabels = ['SFcurve','stcounts','stsem','NOri','NSpf','SpatOris','SpatFrqs',
                  'tXX','tempker','sgcounts','sgsem','Ori','OI','SFI','SpatFreq',
                  'ymean','uu','su','otune','sotune','stune','sstune','MyTick','MyTickLabel',
                  'mcounts','msem','pcounts','psem','tcrit','trepa','trepb','FileTag']
    hart_dict = dict(zip(hartlabels, list(unit_dict['Hart'][0][0])))

    useinds = mRaster(axs[0,uPos], sacim_dict['StimRast2'], 500)
    axs[0,uPos].set_ylabel('gaze shifts')
    
    axs[0,uPos].set_title('cell {}'.format(uPos+1))

    axs[1,uPos].plot(sacim_dict['StimTT'].flatten(), sacim_dict['StimUU'].flatten(), 'k-')
    axs[1,uPos].vlines(0,0,np.max(sacim_dict['StimUU'].flatten())*1.1, 'k', linestyle='dashed',linewidth=1)
    axs[1,uPos].set_ylabel('sp/s')
    axs[1,uPos].set_xticks(np.linspace(-.2,.4,4))
    axs[1,uPos].set_xticklabels(np.linspace(-200,400,4).astype(int))
    axs[1,uPos].set_xlabel('time (ms)')

    axs[2,uPos].plot(hart_dict['SpatOris'].flatten(),
                  np.array(hart_dict['otune'].flatten())[:int(np.size(hart_dict['SpatOris'].flatten()))],
                  color='k')
    axs[2,uPos].set_xticks(hart_dict['SpatOris'].flatten()[::2].astype(int))
    axs[2,uPos].set_xlabel('orientation (deg)')
    axs[2,uPos].set_ylabel('sp/s')

fig.tight_layout()

fig.savefig(os.path.join(savepath, '6_example_cells.pdf'))

In [ ]:
def normalize_psth(psth):
    pref = psth.copy()
    bsln = np.mean(psth[0:150]) # was -100 to -50 ms ; now, -200 to -50 ms
    norm_psth = (psth - bsln) / np.max(pref[200:]) # 0 to 200
    return norm_psth

# Cross validation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
from tqdm import tqdm

def calc_PSTH(spikeT, eventT, bandwidth=10, resample_size=1, edgedrop=15, win=1000, num_events=None):
    """
    calcualtes for a single cell at a time

    bandwidth (in msec)
    resample_size (msec)
    edgedrop (msec to drop at the start and end of the window so eliminate artifacts of filtering)
    win = 1000msec before and after
    """

    # Some conversions
    bandwidth = bandwidth / 1000
    resample_size = resample_size / 1000
    win = win / 1000
    edgedrop = edgedrop / 1000
    edgedrop_ind = int(edgedrop / resample_size)

    # Setup time bins. The bins of the returned PSTH will be -`win` msec to +`win` msec
    # with `resame_size` msec bins, where 0 is the time of the event. Because of edge effects
    # from the gaussian filter, we'll calculate the PSTH with an extra `edgedrop` msec at the
    # start and end so that the edges of the PSTH can be dropped without losing timepoints
    # we care about.
    bins = np.arange(-win-edgedrop, win+edgedrop+resample_size, resample_size)

    # if there is only one event time (i.e. if it's an int for all spikes like eventT=np.array(0)
    if np.size(eventT)>1:

        # Get the timestamps of spikes relative to events in `eventT`
        sps = []
        for i, t in enumerate(eventT):
            sp = spikeT-t
            # Only keep spikes in this window
            sp = sp[(sp <= (win+edgedrop)) & (sp >= (-win-edgedrop))] 
            sps.extend(sp)
        num_events = np.size(eventT)
    else:
        sps = eventT.copy().flatten()
    
    # If `win` is 1000 msec, values in `sps` will fall between -1 and 1, since only spikes
    # that fall before or after the event by 1 second are included.
    # (Because of `edge_drop`, values can be a bit beyond `win` but those will be eliminated before
    # the PSTH is returned.
    sps = np.array(sps)

    # Calculate the PSTH using Kernel Density Estimation
    kernel = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(sps[:, np.newaxis])
    density = kernel.score_samples(bins[:, np.newaxis])

    # Here, `density` is a probability function that sums to 1. We want it in sp/sec.
    # We need to multiply by the # spikes to get the spike count per point. Then,
    # we divide by the number of events so that we get teh rate per event.
    psth = np.exp(density) * (np.size(sps) / num_events)

    # Drop the extra time at the start and end of the window, since it will have
    # edge effects from the gaussian filter. After dropping this, it will be the
    # size set by `win`, so no important timepoints are actually lost here.
    psth = psth[edgedrop_ind:-edgedrop_ind]

    return psth

def normalize_psth(psth, raw_pref=None, baseline_val=None):
    if raw_pref is None:
        raw_pref = psth.copy()
    if baseline_val is None:
        baseline_val = np.mean(psth[0:800].astype(float))
    norm_psth = (psth - baseline_val) / np.max(raw_pref[750:1250].astype(float)) # [1000:1250]
    return norm_psth

In [ ]:
totunits = totdata['TotalInfo']

In [ ]:
uNum = 10

unitlabels = ['tagname','pathname','pathplot','isolation','depth','duration',
              'waveform','channel','shank','ISI','Hart','SacGrating','SacImage']
unit_dict = dict(zip(unitlabels, list(totunits[uNum][0][0][0])))

sacimlabels = ['EventList','StimWin','TrialType','StimRast','OriInd','StimTT',
               'StimUU','StimSU','OriInd2','StimRast2','StimTT2','StimUU2',
               'StimSU2','BaseMu','BaseMu2']
sacim_dict = dict(zip(sacimlabels, list(unit_dict['SacImage'][0][0])))

rast = sacim_dict['StimRast2'].copy()

trials = np.unique(rast[:,1]).astype(int)-1

train_inds = np.array(sorted(np.random.choice(trials, size=int(np.floor(trials.size/2)), replace=False)))
test_inds = trials.copy()
test_inds = np.delete(test_inds, train_inds)

all_sps = []
for tnum in train_inds:
    sps = rast.copy()
    sps = sps[sps[:,1].astype(int)==int(tnum)]
    sps = sps[:,0]
    all_sps.extend(sps)
    
# PSTH
psth = calc_PSTH(np.array(all_sps), np.zeros(train_inds.size), bandwidth=5, win=250)
# normalize
# norm_psth = normalize_psth(psth)

In [ ]:
plt.plot(psth)

In [ ]:
for uPos, uNum in enumerate(601):


In [ ]:
norm_train = np.zeros([n_cells,2001])
norm_test = np.zeros([n_cells,2001])
for i, ind in enumerate(good_inds):
    norm_train[i,:] = normalize_psth(train_psth[ind,:])
    norm_test[i,:] = normalize_psth(test_psth[ind,:])

# Open in arrays

In [ ]:
sacc_psth = data['ISACMOD2']
grat_psth = data['GSACMOD']
sf_tuning = data['SFTUNE']
tf_tuning = data['TFTUNE']
ori_tuning = data['ORTUNE']
bsln_fr = data['BASEMU2']
peakT = data['PEAKIM2']
animal = data['ANIMID']

In [ ]:
n_cells = np.size(peakT,0)

norm_sacc_psth = np.zeros([n_cells, len(psth_bins)])
for ind in range(n_cells):
    norm_sacc_psth[ind,:] = normalize_psth(sacc_psth[ind].copy())

# Grat prefs

In [ ]:
# spatial and temporal frequencies used
sf_vals = np.array([1,2,4,8,16])
tf_vals = (60/16) * np.arange(9)

tf_pref = np.zeros(n_cells); sf_pref = np.zeros(n_cells)
ori_index = np.zeros(n_cells); tf_index = np.zeros(n_cells); sf_index = np.zeros(n_cells);


for ind in range(n_cells):
    sf = sf_tuning[ind,:].copy()
    tf = tf_tuning[ind,:].copy()
    ori = ori_tuning[ind,:].copy()
    
    ofi = np.nanstd(ori) / np.nanmean(ori)
    sfi = np.nanstd(sf) / np.nanmean(sf)
    tfi = np.nanstd(tf) / np.nanmean(tf)
    
    svec = sf.copy()-1
    svec[svec<0] = 0
    svec = svec**2
    spref = np.nansum(svec * sf_vals) / np.nansum(svec)
    
    tvec = tf.copy()-1
    tvec[tvec<0] = 0
    tvec = tvec**2
    tpref = np.nansum(tvec * tf_vals) / np.nansum(tvec)
    
    sf_pref[ind] = spref
    tf_pref[ind] = tpref
    ori_index[ind] = ofi
    sf_index[ind] = sfi
    tf_index[ind] = tfi

In [ ]:
def psth_modind(psth):
    # modulation in terms of spike rate
    psth = psth.astype(float)
    use = psth - np.mean(psth[0:150].copy())
    mod = np.max(np.abs(use[200:]))
    return mod

sacc_resp = np.zeros(n_cells)
mods = np.zeros(n_cells)
for ind in range(n_cells):
    mod = psth_modind(norm_sacc_psth[ind,:])
    mods[ind] = mod
    if mod > 0.1:
        sacc_resp[ind] = True

In [ ]:
pca_input = norm_sacc_psth.copy()

n_pcas = 10
pca = PCA(n_components=n_pcas)
pca.fit(pca_input)

explvar = pca.explained_variance_ratio_

proj = pca.transform(pca_input)

# keep_pcas = int(np.argwhere(np.cumsum(explvar)>.95)[0])
# print('using best {} PCs'.format(keep_pcas))

gproj = proj[:,:7] # 

km = KMeans(n_clusters=4) ### using 4 clusters, not 5...
km.fit_predict(gproj)
Z = km.labels_

In [ ]:
np.cumsum(explvar[:7])

In [ ]:
kord = [0,2,3,1]

color_list = [kcolors['early'],
          kcolors['late'],
          kcolors['biphasic'],
          kcolors['negative']
         ]

In [ ]:
clusters = Z.copy()

In [ ]:
fig, axs = plt.subplots(1,4,figsize=(7.5,2), dpi=300)
for ki, k in enumerate(kord):
    ax = axs[ki]
    inds = np.argwhere(clusters==k).flatten()
    for ind in inds:
        psth = norm_sacc_psth[ind,:]
        ax.plot(psth_bins, psth, alpha=0.2)
    ax.plot(psth_bins, np.nanmean(norm_sacc_psth[inds,:], axis=0), color=color_list[ki], linewidth=2)
    ax.set_ylim([-0.75,1])
    ax.vlines(0, -1, 1, color='k', linestyle='dashed')
    ax.set_xticks(np.linspace(-200,400,4))
    ax.set_xlim([-200,400])
    ax.set_xlabel('time (ms)')
    if ki>0:
        ax.set_yticks([])
    if ki==0:
        ax.set_ylabel('norm sp/s')
fig.tight_layout()
fig.savefig(os.path.join(savepath, '6_clusters.pdf'))

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(4,3.25), dpi=300)
for ki, k in enumerate(kord):
    ax = axs.flatten()[ki]
    inds = np.argwhere(clusters==k).flatten()
    for ind in inds:
        psth = norm_sacc_psth[ind,:]
        ax.plot(psth_bins, psth, alpha=0.2)
    ax.plot(psth_bins, np.nanmean(norm_sacc_psth[inds,:], axis=0), color=color_list[ki], linewidth=2)
    ax.set_ylim([-0.8,1])
    ax.vlines(0, -1, 1, color='k', linestyle='dashed')
    ax.set_xticks(np.linspace(-200,400,4))
    ax.set_xlim([-200,400])
    if ki<2:
        ax.set_xticklabels([])
    if ki==0 or ki==2:
        ax.set_ylabel('norm sp/s')
    if ki>1:
        ax.set_xlabel('time (ms)')
fig.tight_layout()
fig.savefig(os.path.join(savepath, '6_clusters_sq.pdf'))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(2.75,2), dpi=300)
for ki, k in enumerate(kord):
    inds = np.argwhere(clusters==k).flatten()
    for ind in inds:
        psth = norm_sacc_psth[ind,:]
        # ax.plot(psth_bins, psth, alpha=0.2)
    ax.plot(psth_bins, np.nanmean(norm_sacc_psth[inds,:], axis=0), color=color_list[ki], linewidth=2)
    ax.set_ylim([-.5,.7])
    ax.vlines(0, -1, 1, color='k', linestyle='dashed')
    
ax.set_xlim([-200,400])
ax.set_xticks(np.linspace(-200,400,4))
ax.set_xlabel('time (ms)')
ax.set_ylabel('norm sp/s')

fig.tight_layout()
fig.savefig(os.path.join(savepath, '6_all_clusters.pdf'))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(2.5,2), dpi=300)
for ki, k in enumerate(kord):
    inds = np.argwhere(clusters==k).flatten()
    ax.scatter(proj[inds,0], proj[inds,1], s=1, c=color_list[ki])
ax.set_xlabel('PC 1'); ax.set_ylabel('PC 2')
ax.axis('equal')
# ax.set_ylim([-3,5])
# ax.set_xlim([-6,8])

fig.savefig(os.path.join(savepath, '6_pca.pdf'))

In [ ]:
def plot_cprop_scatter(ax, data, clusters, color_list):
    for ki, k in enumerate(kord):
        
        inds = np.argwhere(clusters==k).flatten()
        kdata = data[inds].copy()
        
        x_jitter = np.random.uniform(ki-0.2, ki+0.2, np.size(kdata,0))
        
        ax.plot(x_jitter, kdata, '.', color=color_list[ki], markersize=2)
        
        hline = np.nanmedian(kdata)
        err = np.std(kdata) / np.sqrt(np.size(kdata))
        
        ax.hlines(hline, ki-0.2, ki+0.2, color='k', linewidth=2)
        ax.vlines(ki, hline-err, hline+err, color='k', linewidth=2)
        
        ax.set_xticks(range(4))
        ax.set_xticklabels(['early','late','biphasic','negative'])

In [ ]:
from scipy import stats
def running_median(ax, x, y, n_bins=7):
    bins = np.linspace(np.min(x), np.max(x), n_bins)
    bin_means, bin_edges, bin_number = stats.binned_statistic(x[~np.isnan(x) & ~np.isnan(y)], y[~np.isnan(x) & ~np.isnan(y)], statistic=np.median, bins=bins)
    bin_std, _, _ = stats.binned_statistic(x[~np.isnan(x) & ~np.isnan(y)], y[~np.isnan(x) & ~np.isnan(y)], statistic=np.nanstd, bins=bins)
    hist, _ = np.histogram(x[~np.isnan(x) & ~np.isnan(y)], bins=bins)
    tuning_err = bin_std / np.sqrt(hist)
    ax.plot(bin_edges[:-1] + (np.median(np.diff(bins))/2), bin_means, '-', color='k')
    ax.fill_between(bin_edges[:-1] + (np.median(np.diff(bins))/2), bin_means-tuning_err, bin_means+tuning_err, color='k', alpha=0.2)

In [ ]:
grat_resp = np.zeros(n_cells)
for ind in range(n_cells):
    ffi = np.sqrt(ori_index[ind]**2 + sf_index[ind]**2)
    if ffi >= 0.2:
        grat_resp[ind] = True
grat_resp = grat_resp.astype(bool)

In [ ]:
fig, [[ax0,ax1],[ax2,ax3]] = plt.subplots(2,2, figsize=(6,4.5), dpi=300)

# use = np.argwhere((grat_resp)).flatten()
use = np.argwhere((grat_resp) * (peakT.flatten()>25) * (peakT.flatten()<160)).flatten() # 25; 175 ms

plot_cprop_scatter(ax0, tf_pref[use], clusters[use], color_list)
ax0.set_ylabel('temporal freq (cps)')

plot_cprop_scatter(ax2, sf_pref[use], clusters[use], color_list)
ax2.set_ylabel('spatial freq (cpd)')

for ki, k in enumerate(kord):
    inds = np.intersect1d(np.argwhere((clusters==k)).flatten(), use)
    
    ax1.plot(peakT.flatten()[inds], tf_pref[inds], '.', color=color_list[ki], markersize=3)
    ax3.plot(peakT.flatten()[inds], sf_pref[inds], '.', color=color_list[ki], markersize=3)
    
running_median(ax1, peakT.flatten()[use], tf_pref[use], n_bins=6)
running_median(ax3, peakT.flatten()[use], sf_pref[use], n_bins=6)

ax1.set_xlabel('saccade latency (ms)')
ax3.set_xlabel('saccade latency (ms)')
ax1.set_ylabel('temporal freq (cps)')
ax3.set_ylabel('spatial freq (cpd)')
    
ax1.set_xlim([25,175])
ax3.set_xlim([25,175])
ax0.set_ylim([0, 15]); ax1.set_ylim([0, 15])
ax2.set_ylim([1.5, 5]); ax3.set_ylim([1.5, 5])

fig.tight_layout()

fig.savefig(os.path.join(savepath, '6_sf_tf_scatter.pdf'))

In [ ]:
from scipy.stats import linregress

In [ ]:
x = peakT.flatten()[use]
y = sf_pref[use]

res = linregress(x, y)
(res.rvalue)**2

In [ ]:
(res.rvalue)**2

In [ ]:
res.pvalue

In [ ]:
len(peakT.flatten()[grat_resp])

In [ ]:
use = np.argwhere((grat_resp) * (peakT.flatten()>25) * (peakT.flatten()<150)).flatten()
np.unique(clusters[use], return_counts=True)

In [ ]:
len(use)

In [ ]:
latency_sort = np.argsort(peakT.flatten().copy())
tempseq = norm_sacc_psth.copy()[latency_sort]

In [ ]:
def plot_tempseq(ax, seq):
    
    ax.set_xlabel('time (msec)')
    ax.set_ylim([np.size(seq,0),0])
    
    vmin = -0.75; vmax = 0.75
    img = ax.imshow(seq, cmap='coolwarm', vmin=vmin, vmax=vmax)
    
    ax.set_xlim([0,601])
    
    ax.set_xticks(np.linspace(0,600,4))
    ax.set_xticklabels(labels=np.linspace(-200,400,4).astype(int))
    ax.vlines(200, 0, np.size(seq,0), color='k', linestyle='dashed', linewidth=1)
    
    ax.set_aspect(3)
    
    return img

In [ ]:
fig, ax = plt.subplots(1,1, dpi=300, figsize=(5,5))
plot_tempseq(ax, tempseq)
ax.set_yticks(np.arange(0,np.size(tempseq,0),100))
fig.savefig(os.path.join(savepath, '6_tempseq.pdf'))

In [ ]:
0,2,3,1

In [ ]:
k_to_name = {
    0:'early',
    2:'late',
    3:'biphasic',
    1:'negative'
}

In [ ]:
sorted_clusters = clusters.copy()[latency_sort]

tempseq_legend = np.zeros([n_cells, 1, 4])
for ki, k in enumerate(sorted_clusters):
    tempseq_legend[ki,:,:] = kcolors[k_to_name[k]]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(0.5,1.5), dpi=300)
ax.imshow(tempseq_legend, aspect=.05)
ax.set_yticks([]); ax.set_xticks([])
ax.axes.spines.bottom.set_visible(False)
ax.axes.spines.right.set_visible(False)
ax.axes.spines.left.set_visible(False)
ax.axes.spines.top.set_visible(False)
plt.tight_layout()
fig.savefig(os.path.join(savepath, '6_tempseq_legend.pdf'))

In [ ]:
fig, [ax_cellcounts, ax_baseline_fr] = plt.subplots(1,2, figsize=(5.5,2.5), dpi=300)

names = ['early','late','biphasic','negative']
print_names = ['early','late','biph','neg']

for ki, k in enumerate(kord):
    inds = np.argwhere((clusters==k)).flatten()
    ax_cellcounts.bar(ki, len(inds)/n_cells, color=color_list[ki])
    
ax_cellcounts.set_xticks(ticks=range(4))
ax_cellcounts.set_xticklabels(print_names, rotation=90)
ax_cellcounts.set_ylabel('frac. cells')

for ki, k in enumerate(kord):
    inds = np.argwhere((clusters==k)).flatten()
    bsln_plot_vals = bsln_fr[inds]
    
    err = np.nanstd(bsln_plot_vals) / np.sqrt(np.size(bsln_plot_vals))
    med = np.nanmedian(bsln_plot_vals)
    
    ax_baseline_fr.bar(ki, med, color=color_list[ki])
    ax_baseline_fr.plot([ki, ki], [med-err, med+err], 'k-')
    
ax_baseline_fr.set_xticks(range(4))
ax_baseline_fr.set_xticklabels(print_names, rotation=90)
ax_baseline_fr.set_ylabel('baseline (sp/s)')

fig.tight_layout()
fig.savefig(os.path.join(savepath, '6_ksize_bslnFr.pdf'))